In [2]:
import requests
import zipfile
import os
import json

# URL du fichier ZIP
url = "https://opendata.paris.fr/api/datasets/1.0/comptages-routiers-permanents-historique/attachments/opendata_txt_2020_zip/"

# Nom local du fichier ZIP
zip_filename = "comptages_routiers_2020.zip"

# Dossier où extraire les fichiers
output_folder = "extracted_data"

# Étape 1 : Télécharger le fichier ZIP
response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(zip_filename, "wb") as f:
        f.write(response.content)
    print(f"Fichier téléchargé : {zip_filename}")
else:
    print(f"Erreur lors du téléchargement : {response.status_code}")
    exit()

# Étape 2 : Extraire le contenu du fichier ZIP
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    all_files = zip_ref.namelist()[:2]  # Limiter à 2 fichiers
    for file in all_files:
        zip_ref.extract(file, output_folder)
    print(f"Deux premiers fichiers extraits : {all_files}")

# Étape 3 : Convertir les fichiers texte en JSON
for file in all_files:
    file_path = os.path.join(output_folder, file)
    if file.endswith(".txt"):  # Assurez-vous que le fichier est un fichier texte
        try:
            json_data = []
            with open(file_path, "r", encoding="utf-8") as f:
                # Lire le fichier ligne par ligne et convertir en JSON
                for line in f:
                    data = line.strip().split("\t")  # Séparer les données par tabulation
                    json_data.append(data)
            
            # Sauvegarder les données en JSON
            json_filename = file_path.replace(".txt", ".json")
            with open(json_filename, "w", encoding="utf-8") as json_file:
                json.dump(json_data, json_file, indent=4)
            print(f"Fichier converti en JSON : {json_filename}")
        except Exception as e:
            print(f"Erreur lors du traitement du fichier {file}: {e}")



Fichier téléchargé : comptages_routiers_2020.zip
Deux premiers fichiers extraits : ['trafic_capteurs_2020_W00_20200101_20200108.txt', 'trafic_capteurs_2020_W01_20200108_20200115.txt']
Fichier converti en JSON : extracted_data/trafic_capteurs_2020_W00_20200101_20200108.json
Fichier converti en JSON : extracted_data/trafic_capteurs_2020_W01_20200108_20200115.json


Importation de la table référentiel en csv


In [2]:
import requests

# URL du fichier CSV que vous voulez télécharger
url = 'https://parisdata.opendatasoft.com/api/explore/v2.1/catalog/datasets/referentiel-comptages-routiers/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B'

# Nom local sous lequel vous voulez sauvegarder le fichier
local_filename = 'referentiel.csv'

# Télécharger le fichier
response = requests.get(url)

# Vérifier si la demande a réussi (status_code 200)
if response.status_code == 200:
    with open(local_filename, 'wb') as f:
        f.write(response.content)
    print(f"Fichier téléchargé et sauvegardé sous : {local_filename}")
else:
    print(f"Erreur lors du téléchargement du fichier : {response.status_code}")

Fichier téléchargé et sauvegardé sous : referentiel.csv


Visualisation des tables 

In [3]:
import pandas as pd


# Charger le fichier du référentiel avec les informations géographiques
geo_df = pd.read_csv("referentiel.csv", delimiter=";")
geo_df[['lat', 'lon']] = geo_df['geo_point_2d'].str.split(',', expand=True)
geo_df = geo_df[['Identifiant arc','lat', 'lon']]
# Afficher les premières lignes pour vérifier la structure
geo_df.head(10)

,Identifiant arc,lat,lon
0,6799,48.88610804018005,2.3060170797614274
1,6839,48.882846124767354,2.309608336489109
2,6381,48.88185021600981,2.313636877615184
3,1240,48.867653082202935,2.362827833173667
4,5164,48.83748471020789,2.2572392666841106
5,6603,48.82680441725183,2.3040355371176604
6,6593,48.86194217627273,2.31342704120477
7,6813,48.85824539762091,2.314401975386383
8,4382,48.87468482154655,2.303280892510164
9,269,48.86779977023447,2.314350336430965


In [4]:
try:
    trafic_df = pd.read_csv("extracted_data/trafic_capteurs_2020_W00_20200101_20200108.txt", delimiter=";")
    print("Fichier trafic_capteurs_2020_W00_20200101_20200108.txt chargé avec succès")
    
except Exception as e:
    print(f"Erreur lors du chargement du fichier trafic_capteurs_2020_W00_20200101_20200108.txt : {e}")

trafic_df = trafic_df[['iu_ac','k','t_1h']]
trafic_df ['k'] = trafic_df ['k'].fillna(0)
print(trafic_df.head(4))


Fichier trafic_capteurs_2020_W00_20200101_20200108.txt chargé avec succès
   iu_ac    k                 t_1h
0    799  0.0  2020-01-01 01:00:00
1    799  0.0  2020-01-01 02:00:00
2    799  0.0  2020-01-01 03:00:00
3    799  0.0  2020-01-01 04:00:00


In [5]:
geo_df.rename(columns={"Identifiant arc": "iu_ac"}, inplace=True)
geo_df.head(4)

,iu_ac,lat,lon
0,6799,48.88610804018005,2.3060170797614274
1,6839,48.882846124767354,2.309608336489109
2,6381,48.88185021600981,2.313636877615184
3,1240,48.867653082202935,2.362827833173667


In [6]:
# Faire la jointure sur un identifiant commun (par exemple, 'iu_ac')
merged_df = pd.merge(trafic_df, geo_df, on='iu_ac', how='left')



In [7]:
# Afficher les premières lignes du DataFrame fusionné
merged_df.head(2)
print(merged_df.dtypes)

iu_ac      int64
k        float64
t_1h      object
lat       object
lon       object
dtype: object


In [8]:
import geopandas as gpd

# Charger le fichier GeoJSON des arrondissements de Paris
arrondissements_geojson = "arrondissements.geojson"  
arrondissements_gdf = gpd.read_file(arrondissements_geojson)  # Charger les arrondissements


In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon

# Fonction pour créer la grille à partir des latitudes et longitudes
def creer_grille(lat_min, lat_max, lon_min, lon_max, step):
    grille = []
    latitudes = np.arange(lat_min, lat_max, step)
    longitudes = np.arange(lon_min, lon_max, step)
    
    for i in range(len(latitudes) - 1):
        for j in range(len(longitudes) - 1):
            carre = {
                "id": f"{i}_{j}",  # Identifiant unique pour chaque carré
                "lat_min": latitudes[i],
                "lat_max": latitudes[i + 1],
                "lon_min": longitudes[j],
                "lon_max": longitudes[j + 1],
                "lati": latitudes[i] + 0.005/2,  # Centre du carré
                "long": longitudes[j] +  0.005/2,  # Centre du carré
            }
            grille.append(carre)
    return grille

# Fonction pour trouver le carré correspondant à un point
def trouver_carre(lat, lon, grille):
    center_point = Point(lon, lat)
    if pd.isnull(lat) or pd.isnull(lon):  # Vérifier si lat ou lon est NaN
        return None, None, None
    for carre in grille:
        if carre["lat_min"] <= lat < carre["lat_max"] and carre["lon_min"] <= lon < carre["lon_max"]:
            if not arrondissements_gdf.geometry.unary_union.contains(center_point):
                continue  # Ignorer les carrés hors des arrondissements de Paris
            return carre["id"], carre["lati"], carre["long"]
    return None, None, None

# Fonction pour associer le carré à chaque ligne du DataFrame
def associer_carres(df, grille):
    # Appliquer la fonction trouver_carre à chaque ligne
    df[['carre_id', 'lati', 'long']] = df.apply(
        lambda row: pd.Series(trouver_carre(row['lat'], row['lon'], grille)), axis=1
    )
    return df

# Définir les limites de la grille
lat_min, lat_max = 48.81, 48.92
lon_min, lon_max = 2.255, 2.42
step = 0.005  # Taille d'un carré

# Créer la grille
grille = creer_grille(lat_min, lat_max, lon_min, lon_max, step)


# Nettoyer les colonnes lat et lon
merged_df['lat'] = merged_df['lat'].astype(str).str.strip()
merged_df['lon'] = merged_df['lon'].astype(str).str.strip()

# Convertir les colonnes lat et lon en float
merged_df['lat'] = pd.to_numeric(merged_df['lat'], errors='coerce')
merged_df['lon'] = pd.to_numeric(merged_df['lon'], errors='coerce')



# Associer les carrés
merged_df = associer_carres(merged_df, grille)

# Supprimer les lignes où carre_id est NaN
merged_df = merged_df.dropna(subset=['carre_id'])

# Afficher le DataFrame final
print(merged_df.head())


/tmp/ipykernel_92552/3755955189.py:32: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  if not arrondissements_gdf.geometry.unary_union.contains(center_point):


In [1]:
# Regrouper les données par carré et t_1h, puis calculer la moyenne de k
resultats_groupes = merged_df.groupby(['t_1h','carre_id']).agg(
    moyenne_k=('k', 'mean'),
    somme_k=('k', 'sum'),
    count_k=('k', 'count'),
    lati= ('lati', 'mean'),
    long= ('long', 'mean')
).reset_index()

resultats_groupes.head(10)

NameError: name 'merged_df' is not defined

In [114]:
# Sauvegarder les résultats dans un fichier CSV
resultats_groupes.to_csv('moyennes_par_carre_et_t1h.csv', index=False)

print("Moyennes calculées et sauvegardées dans 'moyennes_par_carre_et_t1h.csv'")

Moyennes calculées et sauvegardées dans 'moyennes_par_carre_et_t1h.csv'
